In [1062]:
import pandas as pd
import numpy as np
import re

In [1063]:
import os
path = "Data/"
list_ = os.listdir(path)

In [59]:
# for file in list:
#     data = pd.read_csv(f"{path}+{file}")
#     loc = file.split(".")[0]
#     data = data.drop(['Unnamed: 0'], axis=1)
#

In [1032]:
loc = "arizona"
data = pd.read_csv(f"{loc}.csv")

In [1033]:
data = data.drop(['Unnamed: 0'], axis=1)
data.shape

(165, 1)

In [1034]:
data = data.drop_duplicates(subset=['0'])
data

,0
0,Many swing states should have earlier primarie...
1,"""Look"" | Solmaz Sharif - September 1, 2016 | V..."
2,In September USDA announced $502 million in fu...
3,Arizona Consortium for the Arts September 2022...
4,Potential pet owners in Texas will be able to ...
...,...
159,"@GuntherEagleman Correct again, so it begs the..."
160,"@BraydenCreation My wife at the Grand Canyon, ..."
162,If more money was in the race for Barnes durin...
163,@annalynnfrey Maybe in Arizona they should sta...


In [1036]:
def parse(text):
    text = text.lower()
    return text

In [1037]:
data = data.drop_duplicates()
data.shape

(69, 1)

In [1038]:
# from dateparser.search import search_dates
# print(search_dates(a))
#https://www.kaggle.com/code/redwankarimsony/nlp-101-tweet-sentiment-analysis-preprocessing/notebook

In [1045]:
import nltk
import random

nltk.download('stopwords')

import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
import datefinder

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bharathkumar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [1048]:
stopwords_english = stopwords.words('english')
tokenizer = TweetTokenizer(preserve_case=False,
                           strip_handles=True,
                           reduce_len=True)
stemmer = PorterStemmer()
link_removed_text = []
before_stemming_list = []
after_stemming_list = []
date = []
for tweet in data["0"].tolist():
    # print('Original Tweet: ')
    # print(a)

    # it will remove the old style retweet text "RT"
    tweet2 = re.sub(r'^RT[\s]+', '', tweet)

    # it will remove hyperlinks
    tweet2 = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet2)


    # it will remove hashtags. We have to be careful here not to remove
    # the whole hashtag because text of hashtags contains huge information.
    # only removing the hash # sign from the word
    tweet2 = re.sub(r'#', '', tweet2)

    # it will remove single numeric terms in the tweet.
    tweet2 = re.sub(r'[0-9]', '', tweet2)

    # print('\nAfter removing old style tweet, hyperlinks and # sign')
    # print(tweet2)
    link_removed_text.append(tweet2)

    tweet_tokens = tokenizer.tokenize(tweet2)

    tweets_clean = []
    for word in tweet_tokens: # Go through every word in your tokens list
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            tweets_clean.append(word)
        try:
            tweets_clean.remove(loc.lower())
        except:
            pass
        try:
            tweets_clean.remove("york")
        except:
            pass
    date.append(datefinder(" ".join(tweets_clean)))
    before_stemming_list.append(tweets_clean)
    tweets_stem = []
    for word in tweets_clean:
        stem_word = stemmer.stem(word)  # stemming word
        tweets_stem.append(stem_word)  # append to the list
    after_stemming_list.append(tweets_stem)

In [1049]:
# l = ["a", "b", "c"]
# print(l)
# try:
#     l.remove("a")
# except:
#     pass
# print(l)

In [1050]:
data["removed_links"] = link_removed_text
data["before_stemming"] = before_stemming_list
data["after_stemming"] = after_stemming_list
data["date"] = date

In [1052]:
data_groups_count = {}
tweets_group = {}
for index, row in data.iterrows():
    date = str(row["date"])
    if date in data_groups_count:
        data_groups_count[date].append(row["before_stemming"])
        tweets_group[date].append(row["0"])
    else:
        data_groups_count[date] = [row["before_stemming"]]
        tweets_group[date] = [row["0"]]

In [1053]:
data_groups = {}
for index, row in data.iterrows():
    date = str(row["date"])
    if date in data_groups:
        data_groups[date] += row["before_stemming"]
    else:
        data_groups[date] = row["before_stemming"]

In [1054]:
for each in data_groups:
    print(each, len(data_groups[each]))

2022-09-13 310
2022-09-03 119
2022-09-09 132
2022-09-10 128
2022-09-04 144
2022-09-07 190


In [1055]:
import yake

language = "en"
max_ngram_size = 1
deduplication_threshold = 0.3
numOfKeywords = 50
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)

keywords = {}
def sort(ele):
    return ele[1]

for key, each in data_groups.items():
    keyword = custom_kw_extractor.extract_keywords(' '.join(each))
    temp = []
    for e in keyword:
        temp.append(e)
    temp = sorted(temp, key=sort)
    keywords[key] = temp

In [1056]:
date_final = []
keywords_final = []
tweets_count = []
ids=[]
location = []
tweets_for_event_id = []
tweets_for_events = []
id = 1
for key, each in data_groups.items():
    temp_id = f"{loc}{id}"
    ids.append(temp_id)
    id+=1
    date_final.append(key)
    keywords_final.append(keywords[key])
    tweets_count.append(len(data_groups_count[key]))
    location.append(loc)
    tweets_temp = tweets_group[key]
    tweets_for_event_id += [temp_id] * len(tweets_temp)
    tweets_for_events += tweets_temp


In [1057]:
new_df = pd.DataFrame()
new_df["event_id"] = ids
new_df["location"] = location
new_df["date"] = date_final
new_df["tweets_count"] = tweets_count
new_df["keywords"] = keywords_final
# new_df["tweets_list"] = tweets_for_event

In [1058]:
new_df

,event_id,location,date,tweets_count,keywords
0,arizona1,arizona,2022-09-13,18,"[(home, 0.035978445329520575), (owners, 0.0412..."
1,arizona2,arizona,2022-09-03,11,"[(october, 0.06880760836778534), (end, 0.08492..."
2,arizona3,arizona,2022-09-09,9,"[(night, 0.03586452558037769), (storms, 0.0358..."
3,arizona4,arizona,2022-09-10,10,"[(german, 0.11332151123048782), (reheld, 0.113..."
4,arizona5,arizona,2022-09-04,8,"[(walden, 0.028225766699880157), (grove, 0.028..."
5,arizona6,arizona,2022-09-07,13,"[(republican, 0.054789296127333964), (finchem,..."


In [1059]:
new_df.to_csv(f"processed_data/final_processed_{loc.lower()}_v1.csv", index=False)

In [1060]:
new_df_tweets = pd.DataFrame()
new_df_tweets["event_id"] = tweets_for_event_id
new_df_tweets["tweets"] = tweets_for_events
new_df_tweets.to_csv(f"tweets_list/{loc.lower()}.csv")

In [1061]:
new_df_tweets

,event_id,tweets
0,arizona1,Many swing states should have earlier primarie...
1,arizona1,In September USDA announced $502 million in fu...
2,arizona1,Potential pet owners in Texas will be able to ...
3,arizona1,"According to court documents, Sherman Edward L..."
4,arizona1,Potential pet owners in Texas will be able to ...
...,...,...
64,arizona6,NBC News projects Democrat Adrian Fontes defea...
65,arizona6,Magnus blamed Republican governors in Septembe...
66,arizona6,"@cipola6672 @RepublicanCoryR Oh, look at this ..."
67,arizona6,If more money was in the race for Barnes durin...


In [1065]:
org_data = pd.read_csv("final_dataset_v1.csv")
org_data.columns

Index(['Unnamed: 0', 'event_id', 'location', 'date', 'tweets_count',
       'keywords', 'event_sentiment', 'classification'],
      dtype='object')

In [1066]:
org_data

,Unnamed: 0,event_id,location,date,tweets_count,keywords,event_sentiment,classification
0,0,Washington1,Washington,2022-09-13,13,"[('allies', 0.0370141109748817), ('trafalgar',...",Positive,historical
1,1,Washington2,Washington,2022-09-16,13,"[('corsi', 0.04162912505109767), ('interview',...",Negative,historical
2,2,Washington3,Washington,2022-09-03,8,"[('director', 0.10912854635271647), ('curiousl...",Positive,sport
3,3,Washington4,Washington,2022-09-09,14,"[('maryland', 0.02620209941119737), ('boulevar...",Negative,sport
4,4,Washington5,Washington,2022-09-10,27,"[('george', 0.04167087239681818), ('school', 0...",Positive,business
...,...,...,...,...,...,...,...,...
139,2,Georgia3,Georgia,2022-09-10,12,"[('county', 0.021428847152118378), ('camden', ...",Negative,sport
140,3,Georgia4,Georgia,2022-09-06,13,"[('county', 0.032262332786760446), ('sheriff',...",Negative,sport
141,4,Georgia5,Georgia,2022-09-13,5,"[('israeli', 0.06771232806350562), ('businessm...",Positive,historical
142,5,Georgia6,Georgia,2022-09-16,14,"[('mark', 0.06777092907666397), ('sabbath', 0....",Negative,historical


In [1069]:
req_data = org_data[["event_id", "location", "date"]]

In [1070]:
req_data

,event_id,location,date
0,Washington1,Washington,2022-09-13
1,Washington2,Washington,2022-09-16
2,Washington3,Washington,2022-09-03
3,Washington4,Washington,2022-09-09
4,Washington5,Washington,2022-09-10
...,...,...,...
139,Georgia3,Georgia,2022-09-10
140,Georgia4,Georgia,2022-09-06
141,Georgia5,Georgia,2022-09-13
142,Georgia6,Georgia,2022-09-16


In [1072]:
events_data = pd.read_csv("final_keywords_list.csv")
events_data.columns

Index(['Unnamed: 0', 'event_id', 'keyword', 'weight'], dtype='object')

In [1073]:
events_data

,Unnamed: 0,event_id,keyword,weight
0,0,Washington1,allies,0.037014
1,1,Washington1,trafalgar,0.037211
2,2,Washington1,sousa,0.043994
3,3,Washington1,george,0.057846
4,4,Washington1,spce,0.057846
...,...,...,...,...
5157,5157,Georgia7,traffic,0.397935
5158,5158,Georgia7,possession,0.397935
5159,5159,Georgia7,piece,0.397935
5160,5160,Georgia7,spoke,0.397935


In [1078]:
vals = range(1, req_data.shape[0]+1)

In [1080]:
req_data["id"] = vals

/var/folders/q9/dwl1w3xx50q3vr_byfd3mb580000gn/T/ipykernel_57459/1644377807.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data["id"] = vals


In [1083]:
new_final_keywords = pd.merge(events_data, req_data, on="event_id")

In [1089]:
new_final_keywords.drop(["Unnamed: 0", "event_id"], axis=1)

,keyword,weight,location,date,id
0,allies,0.037014,Washington,2022-09-13,1
1,trafalgar,0.037211,Washington,2022-09-13,1
2,sousa,0.043994,Washington,2022-09-13,1
3,george,0.057846,Washington,2022-09-13,1
4,spce,0.057846,Washington,2022-09-13,1
...,...,...,...,...,...
5157,traffic,0.397935,Georgia,2022-09-15,144
5158,possession,0.397935,Georgia,2022-09-15,144
5159,piece,0.397935,Georgia,2022-09-15,144
5160,spoke,0.397935,Georgia,2022-09-15,144


In [1090]:
new_final_keywords.to_csv("final_keywords_list.csv")

In [1091]:
tweets_list = pd.read_csv("final_tweets_list.csv")

In [1093]:
tweets_list_new = pd.merge(tweets_list, req_data, on="event_id")

In [1095]:
tweets_list_new.drop(["Unnamed: 0", "event_id"], axis=1)

,tweets,location,date,id
0,https://t.co/KwMLUjBum9\n\nThree jail guards i...,Georgia,2022-09-09,138
1,September Complaint: Voting machine testing co...,Georgia,2022-09-09,138
2,A man accused of fatally shooting another pers...,Georgia,2022-09-09,138
3,"Oops…\n\nGA Tech’s defense has been okay, but ...",Georgia,2022-09-09,138
4,Scott Frost bragged about how his new O-Line c...,Georgia,2022-09-09,138
...,...,...,...,...
2038,@Madonna I make intercessions with Heaven for ...,Washington,2022-09-11,14
2039,Remember when Eagles coach Nick Sirianni took ...,Washington,2022-09-11,14
2040,@SovCam @juliegreen52722 @GloryBound_info It w...,Washington,2022-09-11,14
2041,@NWSportsfan1992 @Softykjr These polls are sup...,Washington,2022-09-11,14


In [1096]:
tweets_list_new.to_csv("final_tweets_list.csv")

In [1098]:
org_data["id"] = vals

In [1099]:
org_data

,Unnamed: 0,event_id,location,date,tweets_count,keywords,event_sentiment,classification,id
0,0,Washington1,Washington,2022-09-13,13,"[('allies', 0.0370141109748817), ('trafalgar',...",Positive,historical,1
1,1,Washington2,Washington,2022-09-16,13,"[('corsi', 0.04162912505109767), ('interview',...",Negative,historical,2
2,2,Washington3,Washington,2022-09-03,8,"[('director', 0.10912854635271647), ('curiousl...",Positive,sport,3
3,3,Washington4,Washington,2022-09-09,14,"[('maryland', 0.02620209941119737), ('boulevar...",Negative,sport,4
4,4,Washington5,Washington,2022-09-10,27,"[('george', 0.04167087239681818), ('school', 0...",Positive,business,5
...,...,...,...,...,...,...,...,...,...
139,2,Georgia3,Georgia,2022-09-10,12,"[('county', 0.021428847152118378), ('camden', ...",Negative,sport,140
140,3,Georgia4,Georgia,2022-09-06,13,"[('county', 0.032262332786760446), ('sheriff',...",Negative,sport,141
141,4,Georgia5,Georgia,2022-09-13,5,"[('israeli', 0.06771232806350562), ('businessm...",Positive,historical,142
142,5,Georgia6,Georgia,2022-09-16,14,"[('mark', 0.06777092907666397), ('sabbath', 0....",Negative,historical,143


In [1103]:
org_data = org_data.drop(["Unnamed: 0", "event_id"], axis=1)

In [1104]:
org_data

,location,date,tweets_count,keywords,event_sentiment,classification,id
0,Washington,2022-09-13,13,"[('allies', 0.0370141109748817), ('trafalgar',...",Positive,historical,1
1,Washington,2022-09-16,13,"[('corsi', 0.04162912505109767), ('interview',...",Negative,historical,2
2,Washington,2022-09-03,8,"[('director', 0.10912854635271647), ('curiousl...",Positive,sport,3
3,Washington,2022-09-09,14,"[('maryland', 0.02620209941119737), ('boulevar...",Negative,sport,4
4,Washington,2022-09-10,27,"[('george', 0.04167087239681818), ('school', 0...",Positive,business,5
...,...,...,...,...,...,...,...
139,Georgia,2022-09-10,12,"[('county', 0.021428847152118378), ('camden', ...",Negative,sport,140
140,Georgia,2022-09-06,13,"[('county', 0.032262332786760446), ('sheriff',...",Negative,sport,141
141,Georgia,2022-09-13,5,"[('israeli', 0.06771232806350562), ('businessm...",Positive,historical,142
142,Georgia,2022-09-16,14,"[('mark', 0.06777092907666397), ('sabbath', 0....",Negative,historical,143


In [1105]:
org_data.to_csv("final_dataset_v1.csv")

In [ ]:
org_data
